# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas as hvplot
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

City_ID            City      Lat       Lng  Max Temp  Humidity  \
0          0      ulsteinvik  62.3432    5.8487    279.99        93   
1          1     bayanhongor  46.7167  100.1167    257.54        84   
2          2         iqaluit  63.7506  -68.5145    262.98        85   
3          3         kayenta  36.7278 -110.2546    284.89        30   
4          4  margaret river -33.9500  115.0667    289.28        67   
..       ...             ...      ...       ...       ...       ...   
561      561     vilyuchinsk  52.9306  158.4028    276.30        75   
562      562          muisne   0.6000  -80.0333    296.39        77   
563      563      isla vista  34.4133 -119.8610    295.01        64   
564      564     las lajitas -24.6833  -64.2500    287.33        90   
565      565        mbandaka   0.0487   18.2603    294.17        94   

     Cloudiness  Wind Speed Country        Date  
0           100        3.21      NO  1731370127  
1            28        1.76      MN  1731370128  
2            97        0.89      CA  1731370130  
3             5        1.53      US  1731370131  
4            99        4.72      AU  1731370132  
..          ...         ...     ...         ...  
561          18       16.00      RU  1731370827  
562          45        2.60      EC  1731370829  
563          20        7.72      US  1731370830  
564         100        5.77      AR  1731370711  
565          30        1.53      CD  1731370832  

[566 rows x 10 columns]

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [60]:
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = 'Humidity',
    geo = True,
    frame_width = 1000,
    frame_height = 1400,
    tiles = "OSM",
    scale= 1,
    c = 'City'
)






map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
city_data_df['Temp'] = city_data_df['Max Temp'] - 273.15

city_data_df

bool = (city_data_df['Temp'] > 21) & (city_data_df['Temp'] < 26)

ideal = city_data_df[bool].dropna()

ideal
# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

City_ID               City      Lat       Lng  Max Temp  Humidity  \
8          8         iskushuban  10.2837   50.2300    295.99        83   
13        13  bahia de caraquez  -0.6000  -80.4167    295.73        77   
17        17            veraval  20.9000   70.3667    298.50        48   
20        20               weno   7.4515  151.8468    299.12        94   
24        24             avarua -21.2078 -159.7750    298.18        57   
..       ...                ...      ...       ...       ...       ...   
554      554       mount darwin -16.7725   31.5838    297.37        53   
557      557      visakhapatnam  17.6900   83.2093    299.09        89   
562      562             muisne   0.6000  -80.0333    296.39        77   
563      563         isla vista  34.4133 -119.8610    295.01        64   
565      565           mbandaka   0.0487   18.2603    294.17        94   

     Cloudiness  Wind Speed Country        Date   Temp  
8            18        4.35      SO  1731370138  22.84  
13           30        3.98      EC  1731370144  22.58  
17            1        3.76      IN  1731370149  25.35  
20          100        3.09      FM  1731370154  25.97  
24           75        6.69      CK  1731370158  25.03  
..          ...         ...     ...         ...    ...  
554          50        0.98      ZW  1731370817  24.22  
557          40        3.09      IN  1731370820  25.94  
562          45        2.60      EC  1731370829  23.24  
563          20        7.72      US  1731370830  21.86  
565          30        1.53      CD  1731370832  21.02  

[124 rows x 11 columns]

### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal[['City','Country','Lat','Lng','Humidity']].copy()

hotel_df['Hotel Name'] = ""

hotel_df


City Country      Lat       Lng  Humidity Hotel Name
8           iskushuban      SO  10.2837   50.2300        83           
13   bahia de caraquez      EC  -0.6000  -80.4167        77           
17             veraval      IN  20.9000   70.3667        48           
20                weno      FM   7.4515  151.8468        94           
24              avarua      CK -21.2078 -159.7750        57           
..                 ...     ...      ...       ...       ...        ...
554       mount darwin      ZW -16.7725   31.5838        53           
557      visakhapatnam      IN  17.6900   83.2093        89           
562             muisne      EC   0.6000  -80.0333        77           
563         isla vista      US  34.4133 -119.8610        64           
565           mbandaka      CD   0.0487   18.2603        94           

[124 rows x 6 columns]

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{hotel_df.loc[index,'Lng']},{hotel_df.loc[index,'Lat']},{radius}"
    params["bias"] = f"proximity:{hotel_df.loc[index,'Lng']},{hotel_df.loc[index,'Lat']}"

    # Set base URL
    base_url = f"https://api.geoapify.com/v2/places" 

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data


Starting hotel search
iskushuban - nearest hotel: No hotel found
bahia de caraquez - nearest hotel: No hotel found
veraval - nearest hotel: Shyam
weno - nearest hotel: High Tide Hotel
avarua - nearest hotel: Paradise Inn
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
saint-joseph - nearest hotel: PALM Hotel & Spa
carnarvon - nearest hotel: No hotel found
kertih - nearest hotel: Hotel Singgah
port mathurin - nearest hotel: Escale Vacances
naze - nearest hotel: ビジネス旅館畠山
mehendiganj - nearest hotel: No hotel found
phnom penh - nearest hotel: Thai chong
zouerate - nearest hotel: فندق تيرس زمور
adamstown - nearest hotel: No hotel found
tres palos - nearest hotel: No hotel found
pizarro - nearest hotel: No hotel found
souillac - nearest hotel: Hotel SkylineInn
arraial do cabo - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
ingeniero guillermo n. juarez - nearest hotel: Hotel El Indiano
puli - nearest hotel: 金山大飯店
san antonio de pale 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [64]:
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    frame_width = 700,
    frame_height = 1400,
    tiles = "OSM",
    scale= 2,
    hover_cols = ['Hotel Name','Country'],
    c = 'City'
)
    

map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)